In [15]:
import tkinter as tk

window = tk.Tk()
window.title("Sudoku")
icon = tk.PhotoImage(file= "Icon.png")
window.iconphoto(True, icon)

import tkinter as tk

canvas = tk.Canvas(window, width=750, height=750, bg="white")
canvas.grid()

case_taille = 50
sudoku_taille = 9 * case_taille

for ligne in range(10): # 9 : nb de lignes
    if ligne % 3 == 0:
        epaisseur = 3
    else:
        epaisseur = 1
    canvas.create_line(50, ligne * case_taille + 50, sudoku_taille + 50, ligne * case_taille + 50, width= epaisseur)

for col in range(10): # 9 : nb de colonne
    if col % 3 == 0:
        epaisseur = 3
    else:
        epaisseur = 1
    canvas.create_line(col * case_taille + 50, 50, col * case_taille + 50, sudoku_taille + 50, width= epaisseur)
def lignes_uniques(grille):
    for ligne in grille:  
        deja_vus = set()  
        for nombre in ligne:  
            if nombre in deja_vus:  
                return False  
            deja_vus.add(nombre)  
    return True  
def colonnes_uniques(grille): 
    for col in range(len(grille[0])): 
        deja_vus = set()
        for ligne in grille: 
            if ligne[col] in deja_vus: 
                return False 
            deja_vus.add(ligne[col])
    return True
def carres_uniques(grille): 
    taille = 3
    for i in range(0, len(grille), taille):
        for j in range(0, len(grille), taille):
            deja_vus = set()
            for k in range(taille):
                for l in range(taille):
                    nombre = grille[i + k][j + l]
                    if nombre in deja_vus:
                        return False
                    deja_vus.add(nombre)
    return True

import time

class Chronometre:
    def __init__(self, window):
        self.window = window
        self.window.title("Chronomètre")
        
        # Variables pour le chronomètre
        self.defile = False
        self.time_ecoule = 0
        self.start_time = 0
        
        # Affichage du temps
        self.time_affiche = tk.Label(self.window, text="0:00", font=("Helvetica", 48))
        self.time_affiche.grid()
        
        # Boutons Start/Stop et Reset
        self.start_stop_button = tk.Button(self.window, text="Démarrer", font=("Helvetica", 16), command=self.lancer)
        self.start_stop_button.grid(pady=10)
        
        self.reset_button = tk.Button(self.window, text="Réinitialiser", font=("Helvetica", 16), command=self.reinitialiser)
        self.reset_button.grid(pady=10)
        
        # Mise à jour du chronomètre
        self.update_timer()
    
    def lancer(self):
        """Démarre ou arrête le chronomètre"""
        if self.defile:
            self.defile = False
            self.start_stop_button.config(text="Démarrer")
        else:
            self.defile = True
            self.start_time = time.time() - self.time_ecoule  # Reprend à partir du temps écoulé
            self.start_stop_button.config(text="Arrêter")
    
    def reinitialiser(self):
        self.defile = False
        self.time_ecoule = 0
        self.time_affiche.config(text="0:00")
        self.start_stop_button.config(text="Démarrer")
    
    def update_timer(self):
        """Met à jour l'affichage du chronomètre"""
        if self.defile:
            self.time_ecoule = time.time() - self.start_time
        minutes = int(self.time_ecoule // 60)
        seconds = int(self.time_ecoule % 60)
        self.time_affiche.config(text=f"{minutes}:{seconds:02d}")
        
        # Mettre à jour toutes les 100 ms
        self.window.after(100, self.update_timer)


# Créer un objet Chronomètre
chronometre = Chronometre(window)

window.mainloop()

TclError: can't use "pyimage15" as iconphoto: not a photo image